<a href="https://colab.research.google.com/github/arrears3/JavaProjects/blob/main/InternCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from itertools import permutations

#create a data frame
file_path = 'schemes_data.xlsx'
df = pd.read_excel(file_path)

# Normalize the values
df['productcost_norm'] = df['Product_cost'].min() / df['Product_cost']
df['deliverytime_norm'] = df['Delivery_time'].min() / df['Delivery_time']
df['productquality_norm'] = df['Product_quality'] / df['Product_quality'].max()

# Print normalized values
print("Normalized DataFrame:")
print(df[['Scheme', 'productcost_norm', 'deliverytime_norm', 'productquality_norm']])

# Define the attributes
attributes = ['productcost_norm', 'deliverytime_norm', 'productquality_norm']

def calculate_weights(ranks):
    m = len(ranks)

    # Calculate the sum of reciprocals of ranks
    sum_reciprocals = [sum(1/r for r in ranks[:t+1]) for t in range(m)]

    # Calculate the normalization factor
    normalization_factor = sum(1/sr for sr in sum_reciprocals)

    # Calculate the weights
    weights = [(1/sr) / normalization_factor for sr in sum_reciprocals]

    return weights

# Generate ranks sets dynamically from 1 to 35 attributes
ranks_sets = [list(range(1, n + 1)) for n in range(2, 36)]  # Starts from 2 to 35

# Array to store weights for each set of ranks
all_weights = []

# Calculate and store weights for each set of ranks in a loop
for ranks in ranks_sets:
    weights = calculate_weights(ranks)
    all_weights.append(weights)

# Function to retrieve weights based on the number of attributes
def get_weights_by_num_attributes(num_attributes, all_weights, ranks_sets):
    for i, ranks in enumerate(ranks_sets):
        if len(ranks) == num_attributes:
            return all_weights[i]
    return None

# Example usage to retrieve weights based on the number of attributes
num_attributes = len(attributes)
base_weights = get_weights_by_num_attributes(num_attributes, all_weights, ranks_sets)


# Function to assign weights based on ranks
def assign_weights(ranks):
    weight_dict = {rank: weight for rank, weight in zip(sorted(ranks), base_weights)}
    return [weight_dict[rank] for rank in ranks]

# Generate all permutations of the ranks [1, 2, 3]
all_permutations = list(permutations([1, 2, 3]))

# Prepare a dictionary to store results
results = {'Scheme': df['Scheme']}

# Loop through each permutation
for perm in all_permutations:
    # Assign weights based on the permutation
    weights = assign_weights(list(perm))

    # Print weights for debugging
    print(f"Weights for permutation {perm}: {weights}")

    # Calculate the total score for each scheme
    total_scores = []
    for i in range(len(df)):
        total_score = sum(df.iloc[i][attr] * weight for attr, weight in zip(attributes, weights))
        total_scores.append(total_score)

    # Print total scores for debugging
    print(f"Total scores for permutation {perm}: {total_scores}")

    # Calculate ranks based on total scores
    ranks = pd.Series(total_scores).rank(ascending=False, method='min').astype(int)

    # Add ranks to results dictionary
    perm_str = '-'.join(map(str, perm))
    results[f'Perm_{perm_str}_Rank'] = ranks.values

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Save the results to an Excel file
results_df.to_excel('sheet.xlsx', index=False)

print("Results have been saved to 'sheet.xlsx'")

Normalized DataFrame:
       Scheme  productcost_norm  deliverytime_norm  productquality_norm
0    Scheme 1          0.980193           0.963930             0.963831
1    Scheme 2          0.940445           1.000000             0.984957
2    Scheme 3          0.989519           0.899099             0.951001
3    Scheme 4          0.933193           0.907512             0.999668
4    Scheme 5          0.956665           0.941801             0.970136
5    Scheme 6          0.948283           0.830203             0.983298
6    Scheme 7          0.968075           0.923081             0.972016
7    Scheme 8          0.984397           0.816360             0.963389
8    Scheme 9          0.961433           0.927933             0.969472
9   Scheme 10          0.934438           0.905471             1.000000
10  Scheme 11          0.940445           0.999911             0.985068
11  Scheme 12          1.000000           0.875002             0.949895
12  Scheme 13          0.991140           

In [2]:
import pandas as pd
import numpy as np

# Load the input Excel file
df = pd.read_excel('scheme_final_ranks_single_sheet.xlsx')

# Initialize a 20x20 matrix with zeros
matrix = np.zeros((20, 20), dtype=int)

# Loop through each scheme
for scheme_idx in range(20):
    scheme = df['Scheme'][scheme_idx]
    # Loop through each permutation column
    for col in df.columns[1:]:
        rank = df[col][scheme_idx] - 1  # Convert rank to zero-indexed
        matrix[scheme_idx, rank] += 1

# Create a DataFrame for the matrix
matrix_df = pd.DataFrame(matrix, columns=[f'Rank {i+1}' for i in range(20)])
matrix_df.insert(0, 'Scheme', [f'Scheme {i+1}' for i in range(20)])

# Save the matrix to an Excel file
matrix_df.to_excel('scheme_final_rank_occurrences.xlsx', index=False)

print("The rank occurrences matrix has been saved to 'scheme_rank_occurrences.xlsx'")

The rank occurrences matrix has been saved to 'scheme_rank_occurrences.xlsx'


In [3]:
import pandas as pd

# Load the input Excel file
df = pd.read_excel('scheme_final_rank_occurrences.xlsx')

# Initialize a list to store cumulative frequencies
cumulative_frequencies = []

# Loop through each scheme
for scheme_idx in range(20):
    scheme_cumulative_frequency = 0

    # Loop through each rank column
    for rank in range(1, 21):
        count = df.iloc[scheme_idx, rank]  # Get the count for the current rank
        scheme_cumulative_frequency += count * ((20 - rank) + 1)

    # Append the cumulative frequency for the current scheme
    cumulative_frequencies.append(scheme_cumulative_frequency)

# Add the cumulative frequencies to the DataFrame
df['Cumulative_Frequency'] = cumulative_frequencies

# Display the results
print(df[['Scheme', 'Cumulative_Frequency']])

# Save the DataFrame with cumulative frequencies to a new Excel file
df.to_excel('scheme_final_cumulative_frequencies.xlsx', index=False)

print("The cumulative frequencies have been calculated and saved to 'scheme_final_cumulative_frequencies.xlsx'")

       Scheme  Cumulative_Frequency
0    Scheme 1                   108
1    Scheme 2                   114
2    Scheme 3                    64
3    Scheme 4                    58
4    Scheme 5                    91
5    Scheme 6                    14
6    Scheme 7                    87
7    Scheme 8                    16
8    Scheme 9                    74
9   Scheme 10                    58
10  Scheme 11                   116
11  Scheme 12                    38
12  Scheme 13                    47
13  Scheme 14                    10
14  Scheme 15                    62
15  Scheme 16                    83
16  Scheme 17                    47
17  Scheme 18                   106
18  Scheme 19                    20
19  Scheme 20                    47
The cumulative frequencies have been calculated and saved to 'scheme_final_cumulative_frequencies.xlsx'


In [4]:
import numpy as np
import pandas as pd
from itertools import permutations

# Read data from the Excel file
file_path = 'schemes_data.xlsx'
df = pd.read_excel(file_path)

# Normalize the values
df['productcost_norm'] = df['Product_cost'].min() / df['Product_cost']
df['deliverytime_norm'] = df['Delivery_time'].min() / df['Delivery_time']
df['productquality_norm'] = df['Product_quality'] / df['Product_quality'].max()

# Print normalized values
print("Normalized DataFrame:")
print(df[['Scheme', 'productcost_norm', 'deliverytime_norm', 'productquality_norm']])

# Define the attributes
attributes = ['productcost_norm', 'deliverytime_norm', 'productquality_norm']

# Function to calculate dynamic weights based on ranks
def calculate_weights(ranks):
    m = len(ranks)
    sum_reciprocals = np.cumsum(1 / np.array(ranks))
    normalization_factor = np.sum(1 / sum_reciprocals)
    weights = (1 / sum_reciprocals) / normalization_factor
    return weights

# Generate all permutations of the ranks for the given number of attributes
num_attributes = len(attributes)
all_permutations = list(permutations(range(1, num_attributes + 1)))

# Prepare a dictionary to store results
results = {'Scheme': df['Scheme']}

# Function to evaluate schemes and calculate ranks based on dynamic weights
def evaluate_schemes(df, attributes, perm):
    # Calculate dynamic weights based on the current permutation
    weights = calculate_weights(list(perm))

    # Calculate the total score for each scheme
    total_scores = df[attributes].dot(weights)

    # Calculate ranks based on total scores
    ranks = total_scores.rank(ascending=False, method='min').astype(int)

    # Return the ranks and weights for the current permutation
    return ranks, weights

# Loop through each permutation
for perm in all_permutations:
    # Evaluate schemes for the current permutation
    ranks, weights = evaluate_schemes(df, attributes, perm)

    # Add ranks to results dictionary
    perm_str = '-'.join(map(str, perm))
    results[f'Perm_{perm_str}_Rank'] = ranks.values

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Save the results to an Excel file
results_df.to_excel('sheet.xlsx', index=False)

print("Results have been saved to 'sheet.xlsx'")


Normalized DataFrame:
       Scheme  productcost_norm  deliverytime_norm  productquality_norm
0    Scheme 1          0.980193           0.963930             0.963831
1    Scheme 2          0.940445           1.000000             0.984957
2    Scheme 3          0.989519           0.899099             0.951001
3    Scheme 4          0.933193           0.907512             0.999668
4    Scheme 5          0.956665           0.941801             0.970136
5    Scheme 6          0.948283           0.830203             0.983298
6    Scheme 7          0.968075           0.923081             0.972016
7    Scheme 8          0.984397           0.816360             0.963389
8    Scheme 9          0.961433           0.927933             0.969472
9   Scheme 10          0.934438           0.905471             1.000000
10  Scheme 11          0.940445           0.999911             0.985068
11  Scheme 12          1.000000           0.875002             0.949895
12  Scheme 13          0.991140           